## Requeriments

In [1]:
import numpy                 as np
import pandas                as pd
import matplotlib.pyplot     as plt
import seaborn               as sns
import missingno as msno
from zipfile import ZipFile as zf
import random
#import sklearn.metrics       as Metrics

from sklearn.impute import SimpleImputer

/pip_envs/Debuggers3.6/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#df = pd.read_csv("/mnt/workstation/data/input_Pesos_Dic_20.csv")#, sep= ",", names =header)
topten = pd.read_csv("/mnt/workstation/data/Input_Topten.csv")
middleten = pd.read_csv("/mnt/workstation/data/Aggregate_20_30.csv")
lastten = pd.read_csv("/mnt/workstation/data/Aggregate_last_10_v2.csv")
labels = pd.read_csv("/mnt/workstation/data/labels.csv")

In [3]:
topten.columns

Index(['id_cli', 'edad', 'mora_max', 'cant_moras_30_ult_12_meses',
       'cant_moras_60_ult_12_meses', 'cant_moras_90_ult_12_meses',
       'cupo_total_tc', 'codigo_ciiu', 'ctas_activas', 'ctas_embargadas',
       'mediana_nom3', 'genero', 'estado_civil', 'nivel_academico',
       'profesion', 'ocupacion', 'tipo_vivienda', 'ciudad_residencia',
       'departamento_residencia', 'ciudad_laboral', 'departamento_laboral',
       'rechazo_credito', 'cat_ingreso', 'rep_calif_cred', 'ind_mora_vigente',
       'cartera_castigada', 'tenencia_tc', 'tiene_consumo', 'tiene_crediagil',
       'tiene_ctas_activas', 'tiene_ctas_embargadas', 'pension_fopep',
       'tiene_cred_hipo_1', 'tiene_cred_hipo_2'],
      dtype='object')

In [4]:
topten = topten.drop(columns=['departamento_residencia',
                                    'ciudad_laboral',
                                    'departamento_laboral',
                                ], axis=0)

In [5]:
middleten.columns

Index(['Unnamed: 0', 'id_cli', 'cant_mora_30_tdc_ult_3m_sf',
       'cant_mora_30_consum_ult_3m_sf', 'periodo', 'ctas_activas',
       'ctas_embargadas', 'cupo_tc_mdo', 'saldo_prom3_tdc_mdo',
       'mediana_nom3_desv', 'mediana_pen3_desv', 'ingreso_nompen_desv',
       'ingreso_final_desv', 'ingreso_nompen', 'ingreso_final',
       'cuota_de_vivienda', 'cuota_de_consumo', 'cuota_rotativos',
       'cuota_de_sector_solidario', 'cuota_sector_real_comercio'],
      dtype='object')

In [6]:
middleten = middleten.drop(columns=['periodo',
                                    'ctas_activas',
                                    'ctas_embargadas',
                                    'saldo_prom3_tdc_mdo',
                                    'cupo_tc_mdo'
                                    ], axis=0)

In [7]:
lastten.columns

Index(['Unnamed: 0', 'id_cli', 'pol_centr_ext', 'saldo_prom3_tdc_mdo',
       'cupo_tc_mdo', 'cant_oblig_tot_sf', 'cant_cast_ult_12m_sr',
       'saldo_no_rot_mdo', 'ind', 'gasto_familiar'],
      dtype='object')

In [8]:
topten.shape

(1047955, 31)

In [9]:
df_Xcli = topten.merge(middleten.iloc[:,1:], on="id_cli", how='left')
df_Xcli.shape

(1047955, 44)

In [10]:
df_Xcli = df_Xcli.merge(lastten.iloc[:,1:], on="id_cli", how='left')
df_Xcli.shape

(1092553, 52)

In [11]:
df_Xcli['mora_max'] = df_Xcli['mora_max'].replace(np.nan,0)
df_Xcli['cant_moras_30_ult_12_meses'] = df_Xcli['cant_moras_30_ult_12_meses'].replace(np.nan,0)
df_Xcli['cant_moras_60_ult_12_meses'] = df_Xcli['cant_moras_60_ult_12_meses'].replace(np.nan,0)
df_Xcli['cant_moras_90_ult_12_meses'] = df_Xcli['cant_moras_90_ult_12_meses'].replace(np.nan,0)

In [12]:
features = pd.DataFrame(df_Xcli.dtypes)
features.columns = ['type']
features_f = features[features['type'] == 'float64'].index
features_o = features[features['type'] == 'object'].index
features_i = features[features['type'] == 'int64'].index
features_b = features[features['type'] == 'bool'].index

In [13]:
for i in features_b:
    df_Xcli[i] = df_Xcli[i].astype('int')

In [14]:
features = pd.DataFrame(df_Xcli.dtypes)
features.columns = ['type']
features_f = features[features['type'] == 'float64'].index
features_o = features[features['type'] == 'object'].index
features_i = features[features['type'] == 'int64'].index
features_b = features[features['type'] == 'bool'].index

In [ ]:
df_Xcli[features_o].describe(include='all').T

In [ ]:
df_Xcli[features_f].describe().T

In [ ]:
sns.set_style('darkgrid')
plt.rcParams.update({'font.size':15})
plt.figure(figsize=(25,10))
sns.heatmap(df_Xcli.isnull(), cbar=False)
plt.title("Top Ten Variables")
plt.xlabel('Variables')
plt.ylabel('rows')

In [ ]:
Nulos = 100*df_Xcli.isnull().sum()/df_Xcli.shape[0]
Nulos = pd.DataFrame(Nulos)
variables_nulas = Nulos[Nulos[0]>30]
variables_nulas

In [ ]:
df_Xcli = df_Xcli.drop(list(variables_nulas.index), axis=1)
df_Xcli.shape

In [ ]:
Nulos = 100*df_Xcli.isnull().sum()/df_Xcli.shape[0]
Nulos

## Missing Values Treatment

In [ ]:
Nulos = pd.DataFrame(Nulos)
Nulos.columns = ['%Null']
Variables_Completar = Nulos[Nulos['%Null']>0]
Variables_Completar

In [ ]:
df_Xcli[Variables_Completar.index]

In [ ]:
features = pd.DataFrame(df_Xcli[Variables_Completar.index].dtypes)
features.columns = ['type']
features_f = features[features['type'] == 'float64'].index
features_o = features[features['type'] == 'object'].index
features_i = features[features['type'] == 'int64'].index
features_b = features[features['type'] == 'bool'].index

## Categorical Variables

In [ ]:
100*df_Xcli[features_o].isnull().sum()/df_Xcli.shape[0]

In [ ]:
imp = SimpleImputer(strategy="most_frequent")
imp.fit(df_Xcli[features_o])
imputed_cat_training = imp.transform(df_Xcli[features_o])
imputed_cat_training

In [ ]:
df_Xcli[features_o] = imputed_cat_training

In [ ]:
100*df_Xcli[features_o].isnull().sum()/df_Xcli.shape[0]

In [ ]:
df_Xcli[features_o].describe(include='all').T

## Numerical variables

In [ ]:
100*df_Xcli[features_f].isnull().sum()/df_Xcli.shape[0]

In [ ]:
means = df_Xcli[features_f].mean(axis=0)
#means.reset_index(inplce = True)
#means.columns = ['Variable','Mean']
means

In [ ]:
for i in features_f:
    ind = df_Xcli[i].isnull().index
    df_Xcli.loc[ind,i] = means.loc[i]

In [ ]:
100*df_Xcli[features_f].isnull().sum()/df_Xcli.shape[0]

In [ ]:
df_Xcli[features_f].describe().T

## Labeling

In [ ]:
labels

In [ ]:
for j in ['ind','gasto_familiar','ingreso_nompen']:
    for i in features_o:
        tag = labels[labels['Var_mean'] ==j]
        tmp = pd.pivot_table(tag[tag['Categoria'].isin(df_Xcli[i].unique())][['Categoria','Label']],
                       values='Label',
                       columns='Categoria').round()
        tmp = tmp.T.to_dict()['Label']
        nueva_variable = i+'_tag_'+j
        df_Xcli[nueva_variable] = df_Xcli[i].map(tmp)

In [ ]:
conditions = [
    (df_Xcli['genero'] == 'M'),
    (df_Xcli['genero'] == 'F')
]
values = [2,1]
df_Xcli['genero_tag'] = np.select(conditions, values)

In [ ]:
df_Xcli.columns

In [ ]:
features = pd.DataFrame(df_Xcli.dtypes)
features.columns = ['type']
features_f = features[features['type'] == 'float64'].index
features_o = features[features['type'] == 'object'].index
features_i = features[features['type'] == 'int64'].index
features_b = features[features['type'] == 'bool'].index

In [ ]:
df_Xcli.shape

In [ ]:
df_Xcli[features_o].describe(include='all').T

In [ ]:
df_Xcli.describe().T.head(60)

In [ ]:
df_Xcli.to_csv("/mnt/workstation/data/dataXid.csv", index=False)